In [ ]:
'''
Planejamento do projeto

#1 - Verificar os arquivos e dados que vou precisar (análise preliminar dos dados)
#2 - Importar todas as bibliotecas necessárias
#3 - Importar e tratar dados
#4 - Fazer loop para criar pastas e planilhas de backup das vendas
#5 - No mesmo loop, calcular os indicadores para cada loja e enviar o e-mail para o respectivo gerente
#6 - Criar ranking de faturamento e enviar e-mail para a diretoria

'''

In [8]:
!pip install tqdm

In [2]:
#Importar bibliotecas e definir caminho dos arquivos
import pandas as pd, win32com.client as win32
from tqdm import tqdm
from pathlib import Path
from datetime import datetime as dt, timedelta

caminho = r'C:\Users\AMCTE\OneDrive\Documentos\Python\Projeto AutomacaoIndicadores'

In [20]:
df_vendas = pd.read_excel(f'{caminho}/Bases de Dados/Vendas.xlsx')
df_emails = pd.read_excel(f'{caminho}/Bases de Dados/Emails.xlsx')
df_lojas = pd.read_csv(f'{caminho}/Bases de Dados/Lojas.csv', encoding='cp1252', sep=';')

display(df_emails.info(), '')
display(df_lojas.head(), '')
display(df_vendas.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26 entries, 0 to 25
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   Loja     26 non-null     object
 1   Gerente  26 non-null     object
 2   E-mail   26 non-null     object
dtypes: object(3)
memory usage: 756.0+ bytes


None

''

,ID Loja,Loja
0,1,Iguatemi Esplanada
1,2,Shopping Midway Mall
2,3,Norte Shopping
3,4,Shopping Iguatemi Fortaleza
4,5,Shopping União de Osasco


''

,Código Venda,Data,ID Loja,Produto,Quantidade,Valor Unitário,Valor Final
0,1,2019-01-01,1,Sapato Estampa,1,358,358
1,1,2019-01-01,1,Camiseta,2,180,360
2,1,2019-01-01,1,Sapato Xadrez,1,368,368
3,2,2019-01-02,3,Relógio,3,200,600
4,2,2019-01-02,3,Chinelo Liso,1,71,71


In [4]:
def calcular_indicador(vendas: pd.DataFrame, loja: str, periodo: str):
    '''Função para calcular os três indicadores para cada loja e para cada período, de acordo com os argumentos fornecidos. A função não retorna nada, apenas calcula os indicadores. Caso o dataframe de vendas esteja vazio, os indicadores serão definidos como zero por padrão

        Parameters
        ----------
        vendas: DataFrame do pandas\n
            O DataFrame que contém os dados das vendas (se for diário, já deve vir filtrado).

        loja: str\n
            A loja para a qual o indicador será calculado

        periodo: str\n
            O período que o indicador será calculado. Deve ser um dos valores 'diario' ou 'anual'
    '''

    if periodo == 'diario' or periodo == 'anual':
        indicadores[periodo]['faturamento'][loja] = vendas['Valor Final'].sum().item() if not vendas.empty else 0
        indicadores[periodo]['diversificação'][loja] = vendas['Produto'].nunique() if not vendas.empty else 0
        indicadores[periodo]['ticket médio'][loja] = (round(vendas['Valor Final'].sum() / vendas['Quantidade'].sum(), 2)).item() if not vendas.empty else 0
    else:
        raise Exception('Argumento inválido fornecido! Insira somente "diario" ou "anual" para o período')

In [5]:
def definir_cenario(valor, meta):
    """
    Retorna um ícone HTML que indica o cenário atual em relação à meta.

    A função compara um valor com a meta correspondente e retorna uma seta verde para indicar que o valor atingiu ou superou
    a meta, ou uma seta vermelha para indicar que o valor está abaixo da meta.

    Parâmetros
    ----------
    valor : float
        O valor atual do indicador.
    meta : float
        O valor da meta a ser comparada.

    Retorna
    -------
    str
        Uma string HTML contendo uma seta verde para cima (se o valor for maior ou igual à meta) ou uma seta vermelha para baixo
        (se o valor for menor que a meta).
    """
    if valor >= meta:
        return '<font color=green>▲</font>'  # Seta para cima (indicador positivo)
    else:
        return '<font color=red>▼</font>'  # Seta para baixo (indicador negativo)


def dict_para_html(loja: str, periodo: str) -> str:
  """
    Converte os dados de indicadores e metas de uma loja para uma tabela HTML formatada.

    A função recebe o nome de uma loja e um período (diário ou anual) e gera uma tabela HTML que exibe os indicadores da loja,
    as metas correspondentes e o cenário atual em comparação às metas. Os dados são preenchidos a partir de dicionários globais
    de indicadores e metas, aplicando formatação monetária e comparação com metas.

    Parâmetros
    ----------
    loja : str
        O nome da loja para a qual os indicadores serão exibidos.
    periodo : str
        O período a ser considerado para os indicadores ('diario' ou 'anual').

    Retorna
    -------
    str
        Uma string contendo a representação da tabela em HTML com os indicadores, metas e cenários.

    Levanta
    -------
    Exception
        Se o nome da loja não for encontrado ou o período for inválido, é levantada uma exceção.

    """
  
  if loja in df_lojas['Loja'].values and (periodo == 'diario' or periodo == 'anual'):
    per = 'Dia' if periodo == 'diario' else 'Ano'
    # Criar tabela html
    tabela_html = '<table border="1" cellpadding="5" cellspacing="0">'
    tabela_html += '<thead><tr>'

    # Adicionar os cabeçalhos da tabela
    tabela_html += '<th>Indicador</th>'
    tabela_html += f'<th>Valor {per}</th>'
    tabela_html += f'<th>Meta {per}</th>'
    tabela_html += f'<th>Cenário {per}</th></tr></thead><tbody>'

    # Preencher a tabela com os valores dos indicadores
    for indicador in indicadores[periodo]:
      tabela_html += '<tr>'
      tabela_html += f'<td>{indicador.title()}</td>'
      tabela_html += f'<td>{cur(indicadores[periodo][indicador][loja])}</td>'
      tabela_html += f'<td>{cur(metas[periodo][indicador])}</td>'
      tabela_html += f'<td>{definir_cenario(indicadores[periodo][indicador][loja], metas[periodo][indicador])}</td>'
      tabela_html += '</tr>'

    tabela_html += '</tbody></table>'
    return tabela_html

  else:
    raise Exception('Argumento inválido fornecido para a função!')

In [18]:
#Obter data mais recente do arquivo
latest_date = max(df_vendas['Data'])
data_completa = latest_date.strftime('%d/%m/%y')
data_abreviada = latest_date.strftime('%d/%m')

#Dicionário de metas
metas = {
    'diario': {
        'faturamento': 1000,
        'ticket médio': 500,
        'diversificação': 4
    },
    'anual': {
        'faturamento': 1650000,
        'ticket médio': 500,
        'diversificação': 120
    }
}

#Dicionário de indicadores
indicadores = {
    'diario': {
        'faturamento': {},
        'ticket médio': {},
        'diversificação': {}
    },
    'anual': {
        'faturamento': {},
        'ticket médio': {},
        'diversificação': {}
    }
}

#Inserir barra de progresso para monitoramento da execução do loop
pbar = tqdm(total=len(df_lojas['ID Loja']), position=0, leave=True)

outlook = win32.Dispatch('outlook.application')

for loja in df_lojas['ID Loja']:
  pbar.update()

  #Filtrar vendas por loja e por data para calcular os indicadores
  nome_loja = df_lojas.loc[df_lojas['ID Loja'] == loja, 'Loja'].item()
  vendas_ano = df_vendas[df_vendas['ID Loja'] == loja]
  vendas_dia = vendas_ano[vendas_ano['Data'] == latest_date]

  calcular_indicador(vendas_dia, nome_loja, 'diario')
  calcular_indicador(vendas_ano, nome_loja, 'anual')

  #Salvar backup das vendas para cada loja. Se não existir uma pasta para a loja, será criada uma
  try:
    Path(f'{caminho}/Backup Arquivos Lojas/{nome_loja}').mkdir()
  except:
    pass
  vendas_ano.to_excel(f'{caminho}/Backup Arquivos Lojas/{nome_loja}/Vendas {nome_loja} - {latest_date.date()}.xlsx')

  #Enviar e-mail para os gerentes
  gerente, email_gerente = df_emails.loc[df_emails['Loja'] == nome_loja, ['Gerente', 'E-mail']].squeeze()

  mail = outlook.CreateItem(0)
  mail.display()
  mail.To = email_gerente
  mail.Subject = f'OnePage Dia {data_completa} - Loja {nome_loja}'
  mail.HTMLBody = f'''
Bom dia, {gerente}!<br><br>

O resultado de ontem (dia {data_abreviada}) da loja {nome_loja} foi:<br><br>

{dict_para_html(nome_loja, 'diario')}
<br><br>

{dict_para_html(nome_loja, 'anual')}
<br>

Segue em anexo a planilha com todos os dados para mais detalhes.<br>
Qualquer dúvida, estou à disposição.<br><br>

Atenciosamente,<br>
Fulano
'''
  anexo  = fr'{caminho}/Backup Arquivos Lojas/{nome_loja}/Vendas {nome_loja} - {latest_date.date()}.xlsx'
  mail.Attachments.Add(anexo)
  #mail.Send() Descomente esta linha para enviar o e-mail automaticamente

pbar.close()

100%|██████████| 25/25 [00:00<00:00, 72.57it/s]


In [19]:
print(df_lojas['Loja'].values)

['Iguatemi Esplanada' 'Shopping Midway Mall' 'Norte Shopping'
 'Shopping Iguatemi Fortaleza' 'Shopping União de Osasco'
 'Shopping Center Interlagos' 'Rio Mar Recife' 'Salvador Shopping'
 'Rio Mar Shopping Fortaleza' 'Shopping Center Leste Aricanduva'
 'Ribeirão Shopping' 'Shopping Morumbi' 'Parque Dom Pedro Shopping'
 'Bourbon Shopping SP' 'Palladium Shopping Curitiba'
 'Passei das Águas Shopping' 'Center Shopping Uberlândia'
 'Shopping Recife' 'Shopping Vila Velha' 'Shopping SP Market'
 'Shopping Eldorado' 'Shopping Ibirapuera' 'Novo Shopping Ribeirão Preto'
 'Iguatemi Campinas' 'Shopping Barra']


In [29]:
HTMLBody = f'''
Bom dia, {gerente}!<br><br>

O resultado de ontem (dia {data_abreviada}) da loja {nome_loja} foi:<br><br>

{dict_para_html(nome_loja, 'diario')}
<br><br>

{dict_para_html(nome_loja, 'anual')}
<br>

Segue em anexo a planilha com todos os dados para mais detalhes.<br>
Qualquer dúvida, estou à disposição.<br><br>

Atenciosamente,<br>
Fulano
'''

In [31]:
from pprint import pprint
pprint(indicadores)

{'anual': {'diversificação': {'Bourbon Shopping SP': 120,
                              'Center Shopping Uberlândia': 120,
                              'Iguatemi Campinas': 120,
                              'Iguatemi Esplanada': 120,
                              'Norte Shopping': 120,
                              'Novo Shopping Ribeirão Preto': 120,
                              'Palladium Shopping Curitiba': 120,
                              'Parque Dom Pedro Shopping': 120,
                              'Passei das Águas Shopping': 120,
                              'Ribeirão Shopping': 120,
                              'Rio Mar Recife': 120,
                              'Rio Mar Shopping Fortaleza': 120,
                              'Salvador Shopping': 120,
                              'Shopping Barra': 120,
                              'Shopping Center Interlagos': 120,
                              'Shopping Center Leste Aricanduva': 120,
                              '

In [30]:
from IPython.display import HTML

display(HTML(HTMLBody))

Indicador,Valor Dia,Meta Dia,Cenário Dia
Faturamento,0,1000,▼
Ticket Médio,0,500,▼
Diversificação,0,4,▼
Indicador,Valor Ano,Meta Ano,Cenário Ano
Faturamento,1653103,1650000,▲
Ticket Médio,191.38,500,▼
Diversificação,120,120,▲


In [7]:
#Função lambda para formatar números no padrão brasileiro
cur = lambda num: f'{num:,}'.replace('.', '_').replace(',', '.').replace('_', ',')

def ranking_para_html(dicionario: dict) -> str:
    """
    Função para converter um dicionário ordenado em uma tabela de HTML formatada.

    Recebe um dicionário onde as chaves representam os nomes das lojas e os valores representam os
    respectivos faturamentos. A função gera uma string HTML contendo uma tabela com duas colunas: "Loja" e "Faturamento",
    com uma linha para cada entrada no dicionário.

    Parâmetros
    ----------
    dicionario : dict
        Um dicionário no formato {loja: faturamento}, onde cada chave é o nome de uma loja e o valor é o faturamento.

    Retorna
    -------
    str
        Uma string contendo a representação da tabela em HTML.
    """

    tabela_html = '<table border="1" cellpadding="5" cellspacing="0">'
    tabela_html += '<thead><tr>'

    # Adicionar os cabeçalhos da tabela
    tabela_html += '<th>Loja</th>'
    tabela_html += '<th>Faturamento</th></tr></thead><tbody>'

    # Preenche a tabela com os valores
    for loja in dicionario:
        tabela_html += '<tr>'
        tabela_html += f'<td>{loja}</td>'
        tabela_html += f'<td>{cur(dicionario[loja])}</td>'
        tabela_html += '</tr>'

    tabela_html += '</tbody></table>'
    return tabela_html

In [19]:
#Criar rankings de faturamento e verificar as melhores e piores lojas do dia e do ano
ranking_dia = dict(sorted(indicadores['diario']['faturamento'].items(), key=lambda item: item[1], reverse=True))
ranking_ano = dict(sorted(indicadores['anual']['faturamento'].items(), key=lambda item: item[1],reverse=True))

melhor_diario = [loja for loja, faturamento in ranking_dia.items() if faturamento == max(ranking_dia.values())]
pior_diario = [loja for loja, faturamento in ranking_dia.items() if faturamento == min(ranking_dia.values())]

melhor_anual = [loja for loja, faturamento in ranking_ano.items() if faturamento == max(ranking_ano.values())]
pior_anual = [loja for loja, faturamento in ranking_ano.items() if faturamento == min(ranking_ano.values())]

#Enviar e-mail para a diretoria
outlook = win32.Dispatch('outlook.application')
mail2 = outlook.CreateItem(0)
mail2.display()
mail2.To = df_emails.loc[df_emails['Gerente'] == 'Diretoria', 'E-mail'].item()
mail2.Subject = f'Desempenho Lojas - {data_completa}'
mail2.HTMLBody = f'''
Prezados, bom dia!<br><br>
Segue abaixo os rankings diário e anual do faturamento das lojas (ranking diário referente ao dia {data_abreviada}):

<h3>Ranking Diário:</h3>
{ranking_para_html(ranking_dia)}<br>

{"As melhores lojas do dia foram" if len(melhor_diario) > 1 else "A melhor loja do dia foi"} <b>{", ".join(melhor_diario)}</b>, com <b>R${cur(max(ranking_dia.values()))}</b> de faturamento,
e {"as piores do dia foram" if len(pior_diario) > 1 else "a pior do dia foi"} {", ".join(pior_diario)}, com R${cur(min(ranking_dia.values()))} de faturamento.<br><br>

<h3>Ranking Anual:</h3>
{ranking_para_html(ranking_ano)}<br>

{"Atualmente as melhores lojas do ano são" if len(melhor_anual) > 1 else "Atualmente a melhor loja do ano é"} <b>{", ".join(melhor_anual)}</b>, com <b>R${cur(max(ranking_ano.values()))}</b> de faturamento,
e {"as piores do ano são" if len(pior_anual) > 1 else "a pior do ano é"} {", ".join(pior_anual)}, com R${cur(min(ranking_ano.values()))} de faturamento.<br>

Qualquer dúvida, estou à disposição.<br><br>

Atenciosamente,<br>
Fulano
'''
#mail.Send() Descomente esta linha para enviar o e-mail automaticamente
    

In [39]:
display(HTML(HTMLBody))

Loja,Faturamento
Salvador Shopping,3.950
Novo Shopping Ribeirão Preto,3.400
Center Shopping Uberlândia,2.651
Shopping Eldorado,2.391
Shopping Center Interlagos,1.582
Shopping Recife,1.366
Norte Shopping,1.259
Shopping União de Osasco,1.207
Shopping Vila Velha,937
Bourbon Shopping SP,676
